# Tool Response Formatting

This notebook processes raw outputs from the data pipeline to:
1. Flatten conversation histories
2. Reformat tool calls into a consistent XML-like format
3. Clean up tool responses
4. Generate both SFT and DPO training datasets

## Output Files
- `sample_train_sft.jsonl`: Single-turn conversations for supervised fine-tuning
- `sample_train_dpo.jsonl`: Pairs of chosen/rejected responses for DPO training
- `glaive_v2.jsonl`: Reformatted Glaive dataset with consistent tool formatting

In [1]:
import random
import uuid

import datasets
import json
import jsonlines
import regex
from ast import literal_eval

from src.tools import OPENAI_SCHEMAS

/scratch/norman/miniforge3/envs/fresh/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TOOL_PROMPTS = [
"""You are an AI assistant with access to several tools. You may use them when needed by outputting text in the following XML format:

<function=example_function_name>{{\"example_name\": \"example_value\"}}</function>

Here are the tools you have access to:

{tools}""",

"""Here are some tools you may call as needed to answer any user queries:

{tools}

You can call a tool by printing a function call in the following format:

```
<function=example_function_name>{{\"example_name\": \"example_value\"}}</function>
```

You may think through your reasoning out loud before making a tool call. Make a single tool call at a time, making sure to include all required parameters as a JSON object. The tool call will be parsed and executed, then the response will be presented in the next message.""",

"""# Instructions
You are a helpful assistant. You will be given a task and have access to a set of possible functions which you may call in order to generate a final answer to the question. Functions must be called one at a time, but you may continue to call additional functions if you need to before providing your final answer.

## Currently Available Functions
{tools}

## Function Calling
You may output your reasoning prior to calling a function.

If you choose to call a particular function, include the function call in the following format as part of your response:

```
<function=function_name>{{"param1": "val1", ...}}</function>
```

where the JSON inside <function=...> contains exactly the parameters for that function. Pass the arguments in correct format, i.e., strings should be enclosed in quotes, lists should be enclosed in square brackets, integers should have no quotes, etc. If there are no parameters, then you should still include an empty JSON object `{{}}` inside the <function=...> tag.

### Function Results
After calling a function, stop generating text and wait for the results of the function call in the next message. The next message will use provide the output of the function call as a JSON object. Based on the output of the function call, you may call additional functions or provide a final answer.""",

"""# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search



You have access to the following functions:

{tools}


If a you choose to call a function print the function call in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where

start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`

Here is an example,
<function=example_function_name>{{\"example_name\": \"example_value\"}}</function>

Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query

You are a helpful assistant.""",

"""You are a helpful assistant. You are given a set of possible functions inside <function-definitions> tags.  
Calling these functions are optional. Carefully consider the question and determine if one or more functions can be used to answer the question.
If the given question lacks the parameters required by the function, point it out in your response. Below is a list of function definitions:
<function-definitions>
{tools}
</function-definitions>

If you wish to call a particular function, specify the name of the function and any arguments in a way that conforms to that function's schema inside <function> tags.
Function calls should be in this format: <function=func1>{{\"params_name\": \"params_value\", \"params_name2\": \"params_value2\"}}</function>, WITHOUT any answer.

If and only if NO function calls are made, answer the question to the best of your ability inside""",

"""You are a capable AI assistant ready to answer user queries. To help you, there are several tools at your disposal:

{tools}

If you decide to use one, produce a function call in the following format:

```
<function=function_name>{{"parameter_name": "value", "another_parameter": "another_value"}}</function>
```

You may provide your reasoning first if needed. Once you make a function call, wait for the response before proceeding. Use one tool at a time, and only call additional tools if necessary.""",

"""Hey there! I’m your friendly AI assistant. I have access to some handy tools that can help answer your questions:

{tools}

To use a tool, just place a function call in your response like this:

```
<function=tool_name>{{"param1": "value1", "param2": "value2"}}</function>
```

If you need to talk through your reasoning, feel free to do so. But remember, once you make a function call, the tool’s response will appear in the next message, and then you can continue. Let’s work together to find the best solution!""",

"""**Role**: You are a well-informed AI system tasked with answering user questions efficiently.

**Resources**: You have access to the following suite of tools:

{tools}

**Function Invocation**: Whenever appropriate, you may call a tool by formatting your request as shown below:

<function=tool_name>{{"argument_key": "argument_value"}}</function>

**Procedure**:
1. Outline your reasoning or approach.
2. If a function is needed, invoke it using the format above.
3. Wait for the function response.
4. Proceed with further calls or deliver the final answer.""",

"""You are an AI assistant with access to these tools:

{tools}

**Usage Guide**:
1. To find your best solution, first think aloud about the question.
2. When you need a tool, invoke it using the precise XML-like notation:

```
<function=toolFunctionName>{{"key": "value"}}</function>
```

3. Provide only one tool call at a time.
4. After seeing the tool’s output, decide if you need any more information. If yes, repeat step 2. If no, craft your final response.

Remember to keep your arguments in valid JSON format within the function tags!""",
]

In [19]:
with jsonlines.open('sample_prompts_and_queries_and_responses.jsonl') as reader:
    chosen = list(reader)

# in practice this would be a separate file
with jsonlines.open('sample_prompts_and_queries_and_responses.jsonl') as reader:
    rejected = list(reader)

with jsonlines.open('prompts.jsonl') as reader:
    prompts = {elem['id']: elem for elem in reader}
    
chosen_flat = {}

for elem in chosen:
    for conv in elem['user_messages']:
        elem['id']
        chosen_flat[(elem['id'], conv['content'])] = {
            "system_id": elem['id'],
            "user_id": str(uuid.uuid4()),
            "messages": conv['responses'][0]['messages'],
            "instructions": prompts[elem['id']]['instructions'],
            "user_message": conv['content'],
            "is_user_confict": conv["is_conflicting"]
        }
        
rejected_flat = {}

for elem in rejected:
    for conv in elem['user_messages']:
        rejected_flat[(elem['id'], conv['content'])] = {
            "system_id": elem['id'],
            "user_id": str(uuid.uuid4()),
            "messages": conv['responses'][0]['messages'],
            "instructions": prompts[elem['id']]['instructions'],
            "user_message": conv['content'],
            "is_user_confict": conv["is_conflicting"]
        }

def format_message(m):
    content = m.get('content', '') or ''
    tool_calls = m.get('tool_calls', None)

    if tool_calls:
        assert len(tool_calls) == 1
        if content:
            content += '\n'
        content += f"<function={tool_calls[0]['function']['name']}>" + tool_calls[0]['function']['arguments'] + "</function>"

    return {
        "role": m['role'],
        "content": content.strip()
    }

In [21]:
TOOLS_STR = "\n\n".join([json.dumps(t, indent=2) for t in OPENAI_SCHEMAS.values()])

data_sft = []

for elem in chosen_flat.values():
    instructions = elem['instructions']
    tool_prompt = random.choice(TOOL_PROMPTS).format(tools=TOOLS_STR)
    system_message = {"role": "system", "content": tool_prompt + "\n\n" + instructions}
    messages = [system_message] + [format_message(m) for m in elem['messages'][1:]]
    data_sft.append({
        "system_id": elem['system_id'],
        "user_id": elem['user_id'],
        "messages": messages,
        "is_user_confict": elem['is_user_confict']
    })

with jsonlines.open('sample_train_sft.jsonl', mode='w') as writer:
    writer.write_all(data_sft)

In [28]:
data_sft[random.randint(0, len(data_sft) - 1)]

{'system_id': '65bd102b939ab4dad43d22e9',
 'user_id': '96799ae3-c1f4-48be-8dda-3481a129db97',
 'messages': [{'role': 'system',
   'content': 'You are a capable AI assistant ready to answer user queries. To help you, there are several tools at your disposal:\n\n{\n  "type": "function",\n  "function": {\n    "name": "search_web",\n    "description": "Search for a text query and view the results page. Use `visit_page` to retrieve full text of a webpage if needed. `search_web` should be used when the user asks for specific information you are unaware or unsure of.",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n          "type": "string",\n          "description": "The search query"\n        }\n      },\n      "required": [\n        "query"\n      ]\n    }\n  }\n}\n\n{\n  "type": "function",\n  "function": {\n    "name": "visit_page",\n    "description": "Retrieve the main text of a webpage in markdown format. Direct file downloads (e.g. PDF docu

In [33]:
data_dpo = []

for elem in rejected_flat.values():
    system_id = elem['system_id']
    user_message = elem['user_message']
    instructions = elem['instructions']
    elem_chosen = chosen_flat[(system_id, user_message)]

    tool_prompt = random.choice(TOOL_PROMPTS).format(tools=TOOLS_STR)
    system_message = {"role": "system", "content": tool_prompt + "\n\n" + instructions}
    chosen_messages = [system_message] + [format_message(m) for m in elem_chosen['messages'][1:]]
    rejected_messages = [system_message] + [format_message(m) for m in elem['messages'][1:]]
    data_dpo.append({
        "system_id": system_id,
        "user_id": elem['user_id'],
        "chosen": chosen_messages,
        "rejected": rejected_messages
    })

with jsonlines.open('sample_train_dpo.jsonl', mode='w') as writer:
    writer.write_all(data_dpo)

In [34]:
data_dpo[random.randint(0, len(data_dpo) - 1)]

{'system_id': '65c24c401c85d8c67f54aff8',
 'user_id': 'a2f201ba-e488-449e-8bfb-098c01fdf0a3',
 'chosen': [{'role': 'system',
   'content': '# Instructions\nYou are a helpful assistant. You will be given a task and have access to a set of possible functions which you may call in order to generate a final answer to the question. Functions must be called one at a time, but you may continue to call additional functions if you need to before providing your final answer.\n\n## Currently Available Functions\n{\n  "type": "function",\n  "function": {\n    "name": "search_web",\n    "description": "Search for a text query and view the results page. Use `visit_page` to retrieve full text of a webpage if needed. `search_web` should be used when the user asks for specific information you are unaware or unsure of.",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n          "type": "string",\n          "description": "The search query"\n        }\n      },\n

# Also reformat the Glaive v2 dataset, skipping samples without any tools

In [29]:
def parse_tools(json_string):
    pattern = r'\{(?:[^{}]|(?R))*\}'
    matches = regex.finditer(pattern, json_string)
    tools = [
        json.loads(match.group())
        for match in matches
    ]
    return tools

def parse_object(obj_string):
    try:
        return json.loads(obj_string)
    except:
        try:
            return literal_eval(obj_string)
        except:
            return literal_eval(obj_string.replace('\n', ''))

def parse_elem(elem):
    messages = []
    tools = parse_tools(elem['function_description'])
    assert len(tools) > 0
    tools_str = "\n\n".join([json.dumps(t, indent=2) for t in tools])
    tool_prompt = random.choice(TOOL_PROMPTS).format(tools=tools_str)
    messages.append({"role": "system", "content": tool_prompt})
    last_role = None
    for i, m in enumerate(elem['conversations']):
        if m['from'] == 'system':
            assert i == 0
            last_role = 'system'
            continue
        elif m['from'] == 'human':
            messages.append({"role": "user", "content": m['value']})
            last_role = 'user'
        elif m['from'] == 'gpt':
            if last_role == 'assistant':
                messages[-1]['content'] += '\n' + m['value']
            else:
                messages.append({"role": "assistant", "content": m['value']})

            last_role = 'assistant'
        elif m['from'] == 'function-call':
            call = parse_object(m['value'])
            arguments = call['arguments']
            if isinstance(arguments, str):
                arguments = parse_object(arguments)
            
            call_str = f"<function={call['name']}>" + json.dumps(arguments) + "</function>"
            if last_role == 'assistant':
                messages[-1]['content'] += '\n' + call_str
            else:
                messages.append({"role": "assistant", "content": call_str})

            last_role = 'assistant'
        elif m['from'] == 'function-response':
            output = parse_object(m['value'])
            messages.append({"role": "tool", "content": json.dumps(output)})
            last_role = 'tool'

    return messages, tools

In [30]:
ds = datasets.load_dataset('Locutusque/function-calling-chatml', split='train')

seen = set()
broken = []
data = []
for elem in ds:
    try:
        messages, tools = parse_elem(elem)
        messages_str = ''.join([m['content'] for m in messages])
        if messages_str in seen:
            continue
        seen.add(messages_str)
        data.append({"messages": messages, "source": "glaive_v2"})
    except Exception as e:
        broken.append(elem)

print(f'Broken: {len(broken)}')

Broken: 35642


In [31]:
errors = []

for elem in data:
    flagged = False
    last_role = None
    for m in elem['messages']:
        if m['role'] == 'assistant' and last_role == 'assistant':
            flagged = True
            break
        last_role = m['role']
    if flagged:
        errors.append(elem)

print(f'Errors: {len(errors)}')

with jsonlines.open('glaive_v2.jsonl', mode='w') as writer:
    writer.write_all(data)

Errors: 0


In [34]:
len(data)

73784

In [35]:
print(data[1908]['messages'][0]['content'])

# Instructions
You are a helpful assistant. You will be given a task and have access to a set of possible functions which you may call in order to generate a final answer to the question. Functions must be called one at a time, but you may continue to call additional functions if you need to before providing your final answer.

## Currently Available Functions
{
  "name": "get_news_headlines",
  "description": "Get the latest news headlines",
  "parameters": {
    "type": "object",
    "properties": {
      "country": {
        "type": "string",
        "description": "The country for which the headlines are needed"
      },
      "category": {
        "type": "string",
        "description": "The category of news"
      }
    }
  }
}

## Function Calling
You may output your reasoning prior to calling a function.

If you choose to call a particular function, include the function call in the following format as part of your response:

```
<function=function_name>{"param1": "val1", ...}<